<a href="https://colab.research.google.com/github/ailab-nda/ML/blob/main/iTownpage_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 住所の検索と緯度経度情報の取得
この演習では２箇所のサイトにアクセスします。

- iタウンページ: 施設の住所を知る
- 国土地理院: 住所から緯度経度への変換を行う

上記のサイトには、アクセス方法に違いがあります。

- iタウンページ：Web ブラウザ経由
- 国土地理院：API 経由

技術的には python によるネットワークプログラミングとデータフレームの処理をしているのですが、授業の範囲外なので省略します。

### (1) 必要なライブラリのインストールおよびインポート

インストール（マシンへインストール）

In [ ]:
import os
#os.environ["https_proxy"] = "http://cmproxy.nda.ac.jp:8080"
!pip install selenium folium --trusted-host pypi.python.org --trusted-host files.pythonhosted.org --trusted-host pypi.org

インポート（プログラムにインポート）

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import re
import pandas as pd
import urllib
import urllib.parse
import requests
import json
import time
import folium

### (2) iタウンページへのアクセス
下記ボックス内の search_area, search_word に検索したい言葉と場所を設定してください。

In [ ]:
search_area = '横浜' # 検索した場所
search_word = 'ラーメン' # 検索したい言葉

print('URL:', 'https://itp.ne.jp/search/?mode=list&A1=&A2=&A4=' + search_area + '&TOWN=&C1=&C2=&C3=&C4=&word=' + search_word + '&PG=1')

ブラウザを立ち上げて、ページを巡回・情報の入手を行います。

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(options=options)
driver.implicitly_wait(10)

url = 'https://itp.ne.jp/search/?mode=list&A1=&A2=&A4=' + search_area + '&TOWN=&C1=&C2=&C3=&C4=&word=' + search_word + '&PG=1'
driver.get(url)
res = driver.find_element(By.CLASS_NAME, "condition__ttl")
hits = int(re.findall(r'\d+', res.text)[0])
print("「", search_area, "」に", hits, "件の「", search_word, "」が見つかりました", sep="")

df = pd.DataFrame()
names = []
addresses = []
stations = []

for i in range(1, hits//20 + 2):
    print('i =', i)
    url = 'https://itp.ne.jp/search/?mode=list&A1=&A2=&A4=' + search_area + '&TOWN=&C1=&C2=&C3=&C4=&word=' + search_word + '&PG=' + str(i)
    print(url)
    driver.get(url)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    #for j in range(300):
    #    driver.execute_script('window.scrollBy(0, 20);')
    time.sleep(1)

    elements1 = driver.find_elements(By.CLASS_NAME, 'result-item-head__ttl')
    elements2 = driver.find_elements(By.CLASS_NAME, 'result-item-cts-desc__area')
    for j in range(len(elements1)):
        names.append(elements1[j].text)
        tmp = elements2[j].text.split('\n')
        addresses.append(tmp[0])
        if len(tmp) == 2:
            stations.append(tmp[1])
        else:
            stations.append('なし')
driver.close()

df = pd.DataFrame()
df['name'] = names
df['address'] = addresses
df['station'] = stations
df

### (3) 国土地理院 API へのアクセス
住所を送信すると緯度経度が返ってきます。

In [ ]:
# 国土地理院API
GeospatialUrl = 'https://msearch.gsi.go.jp/address-search/AddressSearch?q='

lat_list = []
lon_list = []
for index, row in df.iterrows():
    print('\r', index, '/', hits, '件取得', end='')
    s_quote = urllib.parse.quote(row.address)
    response = requests.get(GeospatialUrl + s_quote)
    try:
        lat_list.append(response.json()[0]['geometry']['coordinates'][1])
        lon_list.append(response.json()[0]['geometry']['coordinates'][0])
    except Exception as e:
        print(e)
    time.sleep(0.2)

df['lat'] = lat_list
df['lon'] = lon_list
df

### (4) 結果をファイルへ出力
このファイルと同じ場所（画面左にあるフォルダの形をしたアイコンをクリックするとファイルが見えます）に「検索場所_検索語.csv」というファイルができるので、これを ArcGIS Pro で使用します。

In [ ]:
df.to_csv(search_area + '_' + search_word + '.csv', index=False, encoding="cp932")

### (5) おまけ
この後の処理は ArcGIS Pro で行いますが、表示だけならここでも可能です。

In [ ]:
import folium

# 地図生成
folium_map = folium.Map(location=[df['lat'].mean(), df['lon'].mean()], zoom_start=12)

# マーカープロット
for i, row in df.iterrows():
    folium.Marker(location=[row['lat'], row['lon']], popup=row['name'], icon=folium.Icon(color='red')).add_to(folium_map)

# 地図表示
folium_map